In [1]:
import pandas as pd
import re

This notebook contains the part of the approach that deals with the filtering of the meetings betweeen those relevant for climate change and those that aren't. The original lists of meetings are in the <code>Input</code> and the output of the analysis is in the <code>Ouptut</code> folder. 

## 1. Algorithm and keywords

Here the core of the solution that does the filtering is presented.

In [2]:

def word_find_exact_match(line: str,words: list) -> list:
    """Finds the given words that are present in a string.

    This function searches the given line and finds the given words that are present
    in the line. The matches of the words in the lines must be exact. The words
    that are present in the line are returned

    Parameters
    ----------
    line : str
        The line to be analyzed
    words: list
        A list of the words to be searched in the line

    Returns
    ------
    list
        A list of the words found in the line
    """

    found = [] # will contain the words present in line
    
    # search each word in the line
    for to_be_searched in words:
        # if there is an exact match
        if re.search(r'\b' + to_be_searched + r'\b', line): 
            found.append(to_be_searched) # save the word that is present in line

    return found

In [3]:
# algorithm that gets all the keywords present in the text (and returns them)
def get_keywords_present(interest_string: str, keywords: list) -> tuple([bool, list]):
    """Finds all the keywords that are present

    This function runs the finding of the words in the given line (by turning this string in lowercases)
    and then determines whether in this line there were words that matched or not. It returns a tuple: the first
    combination is True with the keywords found if there were at least one keyword found; the second is False with an empty
    list if there was no match found.

    Parameters
    ----------
    interest_string : str
        The line to be analyzed
    keywords: list
        A list of the words to be searched in the line

    Returns
    ------
    tuple([bool, list])
        Two outputs:
            * (True, keywords present) if some of the keywords were identified in the line
            * (False, empty list) if none of the keywords were found in the line

    """

    # run the search of the keywords in the line
    keywords_present = word_find_exact_match(interest_string.lower(), keywords)

    # creates a boolean to say if there is a match or not
    are_keywords_present = not len(keywords_present) == 0
    
    # returns the boolean created and the results of the keyword search
    return are_keywords_present, keywords_present

Here I present how to load the keywords for the filtering that are present in the <code>lobbying_climate_keywords.xlsx</code> file and turn them in a list that can be used by the above functions. In this file, each row contains a keyword in the <code>Original keywords</code> column and its synonsyms, related words, different spellings, etc. of that keyword in the <code>synonyms/related</code> column. The final list of keywords will contain the keywords in their lowercase form.

In [4]:
# load the file
keywords_db = pd.read_excel("lobbying_climate_keywords.xlsx", sheet_name="final list", engine="openpyxl")

# remove empty columns
keywords_db = keywords_db[['Original keywords', "synonyms/related"]]

# Put an empty string in the empty cells
keywords_db = keywords_db.fillna("")

keywords_db.head(2)

,Original keywords,synonyms/related
0,climate change,
1,global warming,


In [5]:
# This cell cotnains that code that creats a list that will contain all the keywords on python

keywords = [] # will contain all the keywords

for i in keywords_db.index:
    # get the keywords on the first column
    keywords.append(keywords_db.iloc[i]['Original keywords'])

    # adds all the synonsyms, etc. to the list of keywords
    if not keywords_db.iloc[i]['synonyms/related'] == "":
        keywords.extend([x.strip() for x in keywords_db.iloc[i]['synonyms/related'].split(',')])

# transform all the keywords in lowercase
for i in range(len(keywords)):
    keywords[i] = keywords[i].lower()

keywords[0:10]

['climate change',
 'global warming',
 'greenhouse',
 'ghg',
 'ghgs',
 'greenhouse gases',
 'keystone',
 'renewable energy',
 'renewables',
 'renewable energies']

## 2. Run analysis on meetings

In this section I present how to run the filtering. I present the code and I use it on some sample companies whose data is in the <code>Inptut</code> folder. This data is the one that is directly downloaded from the LobbyFacts website.

In [6]:
def run_analysis(company: str, keywords: list) -> pd.DataFrame:
    """Runs the analysis on the company's meetings.

    This function loads the company's meetings from the "Input" folder and then analyze each meeting's description by
    determining whehter any of the specified keywords are present in the description. It saves the results in the "Output" folder and
    it returns them as well: the output is a pd.DataFrame that contains the meetings' dates, their description, and for each
    of these meetings also presents a boolean that says whether the meeting is relevant or not for climate change (that is, there was
    at least one of the keywords present in the description or not) and the keywords that were found in the description or an empty 
    list if none were found.

    Parameters
    ----------
    company : str
        the name of the company to be analyzed. The file containing its data to analyze is called "company_meetings.csv".
        The output file will be called "company_meetings_filtered.xlsx"
    keywords: list
        The climate change-related keywords

    Returns
    ------
    pd.DataFrame
        It contains the meetings' date, description, whether is relevant or not, and the keywords found or an empty list if
        none were found.

    """
    
    # load the company's meetings
    meetings = pd.read_csv("Input/" + company + "_meetings.csv")

    # keep only the data and description info
    meetings = meetings[['Date','Subject']].copy() 

    # run analysis

    # columns that will contain the results of the analysis
    meetings['Relevant'] = ""
    meetings['Keywords found'] = ""

    # for each meeting
    for i in meetings.index:
        # run the analysis on the description of the meetings
        are_keywords_present, keywords_present = get_keywords_present(meetings.iloc[i]["Subject"], keywords)

        # save the results
        meetings.at[i, 'Relevant'] = are_keywords_present
        meetings.at[i, 'Keywords found'] = keywords_present

    # compute the percentage of relevant meetings over the total meetings
    print("% of relevant meetings: " + str(meetings.loc[meetings["Relevant"] == True].shape[0]/meetings.shape[0]))

    # save the results
    meetings.to_excel("Output/" + company + "_meetings_filtered.xlsx")

    # return the results
    return meetings

In [7]:
# example of running the analysis on a company

#company = "Amazon"
#company = "Uniper"
company = "Volkswagen"

meetings = run_analysis(company, keywords)

% of relevant meetings: 0.11956521739130435


In [10]:
meetings.head(10)

,Date,Subject,Relevant,Keywords found
0,2015-04-09,Decabornisation of Road Transport,False,[]
1,2015-04-21,Decarbonization,True,[decarbonization]
2,2015-04-21,Energy Union and Transport,False,[]
3,2015-04-27,"standards, data, transport",False,[]
4,2015-04-30,Visit to Audi Brussels in frames of European S...,False,[]
5,2015-05-11,DSM,False,[]
6,2015-05-27,sustainable development,False,[]
7,2015-06-09,Automatiertes Fahren + Industrie 4.0,False,[]
8,2015-07-02,"DSM, digitalisation in the car sector",False,[]
9,2015-09-09,Transport policy,False,[]
